# GLSL Frag Shader Renderer Colab

Put together by [Shellworld](https://github.com/shellward). [(@shellworld1)](https://twitter.com/shellworld1)

Largely taken from:
https://colab.research.google.com/github/tensorflow/lucid/blob/master/notebooks/differentiable-parameterizations/appendix/colab_gl.ipynb

But it wasn't working out of the box due to some configuration stuff covered [here](https://github.com/mcfletch/pyopengl/issues/27)

I don't intend to claim this code as my own! I just cobbled this stuff together to make it easier for me to render frag shaders with a cloud GPU. 



In [ ]:
#@title Connect google drive
from google.colab import drive
drive._mount('/content/drive')

In [ ]:
#@title Install required packages
!pip install -q moviepy
!pip uninstall tensorflow -y
!pip install tensorflow==1.15
!pip install tensorflow-gpu==1.15
!pip install lucid

In [ ]:
#@title Imports
import numpy as np
import json
import moviepy.editor as mvp
from google.colab import files
import lucid.misc.io.showing as show

# Modify config to specify OpenGL in egl.py
# https://github.com/mcfletch/pyopengl/issues/27
# /usr/local/lib/python3.7/dist-packages/OpenGL/platform/egl.py
a_file = open("/usr/local/lib/python2.7/dist-packages/OpenGL/platform/egl.py", "r")
list_of_lines = a_file.readlines()
list_of_lines[36] = "                    'OpenGL',\n"
a_file = open("/usr/local/lib/python2.7/dist-packages/OpenGL/platform/egl.py", "w")
a_file.writelines(list_of_lines)
a_file.close() 

In [ ]:
#@title Test that OpenGL Context is available
WIDTH = 1080 #@param {type:"integer"}
HEIGHT = 1080 #@param {type:"integer"}
from lucid.misc.gl.glcontext import create_opengl_context
# Now it's safe to import OpenGL and EGL functions
import OpenGL.GL as gl
# create_opengl_context() creates GL context that is attached to an
# offscreen surface of specified size. Note that rendering to buffers
# of different size and format is still possible with OpenGL Framebuffers.
#
# Users are expected to directly use EGL calls in case more advanced
# context management is required.
create_opengl_context((WIDTH, HEIGHT))
# OpenGL context is available here.
print(gl.glGetString(gl.GL_VERSION))
print(gl.glGetString(gl.GL_VENDOR)) 
print ('Everything looks good!')
#print(gl.glGetString(gl.GL_EXTENSIONS))


In [18]:
#@title Add your shader
shader_code = """
float rand(vec2 co){
    return fract(sin(dot(co.xy ,vec2(12.9898,78.233))) * 43758.5453);
}

float hexSDF(vec2 p, float size){
    float h = size * 0.5;
    float d = abs(p.x - h) + abs(p.y - h);
    return d - h;
}
vec2 translate(vec2 inp, vec2 offset){
    return inp + offset;
}

vec2 rectToPolar(vec2 inp){
    return vec2(atan(inp.y, inp.x), length(inp));
}

vec2 polarToRect(vec2 inp){
    return vec2(sin(inp.x) * inp.y, cos(inp.x) * inp.y);
}

vec2 rotate(vec2 inp, float angle){
    return vec2(inp.x * cos(angle) - inp.y * sin(angle), inp.x * sin(angle) + inp.y * cos(angle));
} 

float generateNoiseField(vec2 p){
    vec2 q = vec2(p.x / iResolution.x, p.y / iResolution.y);
    float n = 0.0;
    n += 0.5 * rand(q + vec2(0.0, 0.0));
    n += 0.25 * rand(q + vec2(0.0, 1.0));
    n += 0.125 * rand(q + vec2(1.0, 1.0));
    n += 0.0625 * rand(q + vec2(1.0, 0.0));
    return n;
}

vec2 slerp(vec2 a, vec2 b, float t){
    float omega = acos(dot(a, b));  
    return sin((1.0 - t) * omega) * a + sin(t * omega) * b;
}

vec2 grid(vec2 p, float size){
    vec2 q = vec2(floor(p.x / size), floor(p.y / size));
    return q * size;
}

float rect(vec2 p, vec2 size){
    vec2 q = grid(p, size.x);
    vec2 r = vec2(p.x - q.x, p.y - q.y);
    float d = min(min(abs(r.x), abs(r.y)), size.y - abs(r.y));
    return d;
}

float circle(vec2 p, float size){
    return length(p) - size;
}
float fill(vec2 p, float size){
    return max(abs(p.x) - size, abs(p.y) - size);
}
void mainImage(){
vec2 pixelSize = vec2(1.0/iResolution.x, 1.0/iResolution.y);
vec2 uv = gl_FragCoord.xy / iResolution.xy;
vec2 p = uv * 2.0 - 1.0;
vec2 center = vec2(0.5, 0.5);
vec3 background = vec3(0.0, 0.0, 0.0);
p-=rotate(sin(p)*cos(center)*vec2(cos(iTime)/5.,sin(iTime)/5.), sin(iTime)*.91);
p*=translate(p, vec2(-.0, 0.0));
p-=translate(p, vec2(cos(iTime/89.0), sin(iTime/9.0)))*.3;
p+=translate(p, vec2(0.0075, 0.005));
p += 1.-sin(p)*fract(grid(p, 2.4));
p+=rectToPolar(p/center)*slerp(vec2(1.0, .0), vec2(0.0, 1.0), sin(iTime/8.0));
p+= fract(grid(p*8., 1.0));
p+=mix(p,grid(fract(p),3.),sin(iTime/8.0));
for (int i=0; i<2; i++){ 
    p+=rectToPolar(rotate(p,iTime*.05) * 00.5);
    p=p+float(i)+sin(p/center);
    p=polarToRect(p-sin(iTime*.01));
    p=translate(p,vec2(0.5,0.5));
    p=rotate(p, iTime*.01*float(i));
    background+= rect(p*float(i), vec2(.01, 1.)) * 0.94;
    background-= circle(p, .1) * 0.04;//*generateNoiseField(p*float(i));
    background-= vec3(p.x, sin(p-center).y*9., p.x/p.y*.09);
    p=rotate(p, iTime*.001);
    background+= circle(p, .9);//*generateNoiseField(p*float(i));
}
background-=normalize(background)*iTime*generateNoiseField(uv);
background=smoothstep(background, vec3(p.x, p.y, p.x/p.y), vec3(center.x,center.y,center.x/center.y)*12. );
float hex = hexSDF(p, .9);
float time = iTime;
uv += rand(uv) * 0.1/vec2(1.0, 1.0);
vec3 color = .88-background;
  color /= vec3(1./hex*sin(time)/rand(uv*4.), 1./hex*cos(time), hex*rand(uv));
    color.rb += slerp(rectToPolar(uv-p), polarToRect(uv-p), sin(time));
    color.b -= rand(uv)/mod(uv.g,3.);
    color.g -= sin(uv.g+time); 
    color += pow(sin(uv.g), 7.);
    color = normalize(color/sin(time));
    color/=normalize(color*color*color);
    color+=normalize((color*color*color));
color = smoothstep(color, vec3(p.x, p.y, p.x*p.y), vec3(center.x,center.y,center.x/center.y)*0.003 );
color -= vec3(0.0549, 0.4863, 0.0)/color/.900;
gl_FragColor=vec4(color,1.0);
}
"""

In [ ]:
#@title Test that shader works
from OpenGL.GL import shaders

vertexPositions = np.float32([[-1, -1], [1, -1], [-1, 1], [1, 1]])
VERTEX_SHADER = shaders.compileShader("""
#version 330
layout(location = 0) in vec4 position;
out vec2 UV;
void main()
{
  UV = position.xy*0.5+0.5;
  gl_Position = position;
}
""", gl.GL_VERTEX_SHADER)

FRAGMENT_SHADER = shaders.compileShader("""
#version 330
out vec4 outputColor;
in vec2 UV;

uniform sampler2D iChannel0;
uniform vec3 iResolution;
vec4 iMouse = vec4(0);
uniform float iTime = 0.0;
""" + shader_code + """
void main()
{
    mainImage();
}

""", gl.GL_FRAGMENT_SHADER)

shader = shaders.compileProgram(VERTEX_SHADER, FRAGMENT_SHADER)
time_loc = gl.glGetUniformLocation(shader, 'iTime')
res_loc = gl.glGetUniformLocation(shader, 'iResolution')

def render_frame(time):
  gl.glClear(gl.GL_COLOR_BUFFER_BIT)
  with shader:
    gl.glUniform1f(time_loc, time)
    gl.glUniform3f(res_loc, WIDTH, HEIGHT, 1.0)
    
    gl.glEnableVertexAttribArray(0);
    gl.glVertexAttribPointer(0, 2, gl.GL_FLOAT, False, 0, vertexPositions)
    gl.glDrawArrays(gl.GL_TRIANGLE_STRIP, 0, 4)
  img_buf = gl.glReadPixels(0, 0, WIDTH, HEIGHT, gl.GL_RGB, gl.GL_UNSIGNED_BYTE)
  img = np.frombuffer(img_buf, np.uint8).reshape(HEIGHT, WIDTH, 3)[::-1]
  return img
show.image(render_frame(10.0)/255.0, format='jpeg')

In [ ]:
#@title Render Video
duration = 60.0 #@param {type:"number"}
fps = 60 #@param {type:"integer"}
clip = mvp.VideoClip(render_frame, duration=duration)
clip.write_videofile('out.mp4', fps=fps)
files.download('out.mp4')

# Save it to google drive

I haven't done any fancy file management here- it just attempts to make a directory at 
>/content/drive/MyDrive/GLSLCloudRenderer/ 

and then saves the file as **out.mp4**. This will be overwritten if you save another shader.

In [24]:
#@title Copy to Google Drive
!mkdir /content/drive/MyDrive/GLSLCloudRenderer
!cp out.mp4 /content/drive/MyDrive/GLSLCloudRenderer/